In [1]:
from collections import Counter

# AMINO ACID LETTER → MASS
AMINO_ACIDS = {
    'G':57, 'A':71, 'S':87, 'P':97, 'V':99,
    'T':101, 'C':103, 'I':113, 'L':113,
    'N':114, 'D':115, 'K':128, 'Q':128,
    'E':129, 'M':131, 'H':137, 'F':147,
    'R':156, 'Y':163, 'W':186
}

# MASS OF PEPTIDE
def peptide_mass(peptide):
    return sum(AMINO_ACIDS[a] for a in peptide)

# LINEAR SPECTRUM
def linear_spectrum(peptide):
    prefix = [0]
    for a in peptide:
        prefix.append(prefix[-1] + AMINO_ACIDS[a])

    spec = [0]
    n = len(peptide)

    for i in range(n):
        for j in range(i+1, n+1):
            spec.append(prefix[j] - prefix[i])

    return sorted(spec)

# CYCLIC SPECTRUM
def cyclic_spectrum(peptide):
    prefix = [0]
    for a in peptide:
        prefix.append(prefix[-1] + AMINO_ACIDS[a])

    total = prefix[-1]
    spec = [0]
    n = len(peptide)

    for i in range(n):
        for j in range(i+1, n+1):
            diff = prefix[j] - prefix[i]
            spec.append(diff)
            if i > 0 and j < n:
                spec.append(total - diff)

    return sorted(spec)

# CONSISTENCY CHECK
def is_consistent(peptide, spectrum_counter):
    pep_spec = Counter(linear_spectrum(peptide))
    for m in pep_spec:
        if pep_spec[m] > spectrum_counter[m]:
            return False
    return True

# EXPAND
def expand(peptides, allowed_letters):
    new_list = []
    for p in peptides:
        for a in allowed_letters:
            new_list.append(p + a)
    return new_list

# MAIN BRANCH & BOUND
def branch_bound_kmers(spectrum):

    spectrum_counter = Counter(spectrum)
    parent_mass = max(spectrum)

    # Step 1: allowed amino acids from spectrum
    allowed_letters = []
    for aa, mass in AMINO_ACIDS.items():
        if mass in spectrum:
            allowed_letters.append(aa)

    print("Allowed 1-mers:", allowed_letters)

    peptides = [""]
    k = 1
    final = []

    while peptides:
        peptides = expand(peptides, allowed_letters)
        next_round = []

        print(f"\nConsistent {k}-mers:")

        for pep in peptides:
            m = peptide_mass(pep)

            if m == parent_mass:
                if Counter(cyclic_spectrum(pep)) == spectrum_counter:
                    print("FINAL:", pep)
                    final.append(pep)

            elif m < parent_mass:
                if is_consistent(pep, spectrum_counter):
                    print(pep)
                    next_round.append(pep)

        peptides = next_round
        k += 1

    return final

# RUN
spectrum = [
0,97,97,99,101,103,196,198,198,200,202,
295,297,299,299,301,394,396,398,400,400,497
]

result = branch_bound_kmers(spectrum)

print("\nFinal Cyclic Peptides:", result)

Allowed 1-mers: ['P', 'V', 'T', 'C']

Consistent 1-mers:
P
V
T
C

Consistent 2-mers:
PV
PT
PC
VP
VT
VC
TP
TV
CP
CV

Consistent 3-mers:
PVT
PVC
PTP
PTV
PCV
VPT
VPC
VTP
VCP
TPV
TPC
TVP
CPV
CPT
CVP

Consistent 4-mers:
PVCP
PTPV
PTPC
PCVP
VPTP
VCPT
TPVC
TPCV
CPTP
CVPT

Consistent 5-mers:
FINAL: PVCPT
FINAL: PTPVC
FINAL: PTPCV
FINAL: PCVPT
FINAL: VPTPC
FINAL: VCPTP
FINAL: TPVCP
FINAL: TPCVP
FINAL: CPTPV
FINAL: CVPTP

Final Cyclic Peptides: ['PVCPT', 'PTPVC', 'PTPCV', 'PCVPT', 'VPTPC', 'VCPTP', 'TPVCP', 'TPCVP', 'CPTPV', 'CVPTP']
